**Implementation**

Our DeltaPi software provides the user with a package he or she can use to evaluate functions of multiple variables at a specified point while at the same time evaluating the partial derivatives of the function at that same point. A class is provided to perform simple calculations, and two functions are provided to handle vector inputs and outputs.
DeltaPi performs automatic differentiation via the algorithim known as forward mode.

Our strategy has been to create input-variable objects in python that once defined may be used in python calculations as if they were standard python variables of type float. However, when calculations are done with these AutDiff objects, a partial derivative with respect to one of the input variables is calculated and carried along. The object representing the result then contains an attribute that contains the value of the function at the specified point (val) and an attribute that contains a partial derivative with respect to one of the input variables (derv). Which partial derivative is calculated is specified by the user by intiating the derv attribute of one input variable to 1 and the derv attributes of the other input variables to 0.  The calculation can then be repeated to get partial derivatives with respect to other input variables by changing which input variable’s derv attribute is intiated to 1. A function *all_derivatives( )* is supplied to automate this process.

*The AutDiff( ) class*

Most of the functionality of our DeltaPi package is implemented in a single class called AutoDiff( ). Code for this class can be found in the module 'AutoDiff.py' in the *code* directory. It has two attributes, self.val and self.derv, that respectively hold the value of the function being evaluated and the partial derivative of the function being evaluated up to some point in the calculation.

Autodiff() has many methods. Each is short and designed to replace a standard mathematical operation. Thus, for example there is a method \_\_add\_\_ that will overload the standard addition operator (+) and thus specify how objects of type AutoDiff will be added to scalar values or to each other. And similar methods are included to overload the other standard mathematical operators (- * / ^ +). Also, methods are included to replace many 
numpy elemenatry functions so that these functions may be used as well in any calculations involving AutoDiff objects. Two functions are included separately outside the AutoDiff class. They are LogN and Logist, as these
functions are not implemented in numpy, they had to overloaded with a diferent approach. A Table of the functions
and operations overloaded in the AutoDiff module is shown below:



| Arithmetic     | Power    | Log     | Urnary  | Trig    | Hyper   | Special  | Comparison |  
| :------------  | :------- | :------ | :------ | :------ | :------ | :------- | :--------: |
| Addition       | $x^{a}$  | log     | neg     | sin     | sinh    | logistic |    >       |
| subtraction    | $a^{x}$  | log2    | pos     | cos     | cosh    |          |    <       |
| Multiplication | $y^{x}$  | log10   |         | tan     | tanh    |          |    =       |
| Division       | $e^{x}$  | logN    |         | arcsin  | arcsinh |          |    <=      |
|                |          |         |         | arccos  | acrcosh |          |    >=      |
|                |          |         |         | arctan  | acrtanh |          |    !=      |

Central to all Autodiff( ) methods is the principle that each method returns a new AutDiff object that will represent the next stage in the function’s evaluation. For example, if x is an AutDiff object, and it is multiplied by the scalar 2 (x\*2), then x’s \_\_mul\_\_ method is called, and it returns a new object that will represent x\*2. The AutoDiff.\_\_mul\_\_ method performs this calculation as follows. It assumes that 2 is another object of type AutoDiff, and trys to multiply the two AutoDiff objects together. It finds that 2 is not an AutoDiff object: it falls to its except block, and then it makes two calculations: 2 * x.val and 2 * x.derv and stores these values in the returned object’s val and derv attributes. If the 2 had been to the right of x (2*x), then the \_\_rmul\_\_ method would have been called, and it would have executed the same operation.  

More subtle is the case where two AutoDiff objects, say x and y, are multiplied together x\*y. Here x’s \_\_mul\_\_ method is called. It finds that the two objects are instances of AutoDiff, and it 1) multiplies x.val and y.val together and stores the result as the returned object’s val attribute, and 2) calculates (x.derv\*y.val + y.derv\*x.val) and store this value as the returned object’s derv attribute. Notice in the second calculation the derivatives from both objects x and y are carried along as a sum. Recall, however, that the derv value of either x or y will have been initiated to 0, so one term in the sum (x.derv\*y.val + y.derv\*x.val) will always evaluate to 0. In this way, the algorithm can automatically choose the proper derivative to carry along for subsequent calculations (here either that of x or that of y) when two Autodiff objects collide. This same approach has been used generally to  overload the division and power operators as well.

An example of a elementary mathematical function implemented in AutoDiff( ) is the exponential function. This works as follows. For a given AutoDiff object, say x, when exp(x) is executed, x’s exp() function is called on itself. It uses numpy’s exp() function to evaluate np.exp(x.val) and store this value in the return object’s val attribute. It then evaluates the derivative of exp(x.value) —which in this case is just exp(x.value)— and stores this value in the return object’s derv attribute. This approach has been used generally for all the elementary functions.

Autdiff objects are fully functional replacements for python float variables such that once the AutoDiff class is defined, these objects can be used to perform forward automatic differentiation on any multi-input, single output function at any specified point. 

*The all_derivatives() function*

This function allows the user to calculate all partial derivatives of a given function at once without having to initiate the derv attributes of the input variables several times manually. It operates by evaluating the desired function once for each input variable and during each run altering which variable's derv attribute is set to 1, while the other's derv attributes are set to 0. It takes as input: the function to be evaluated, a list of independent variables, and the values of these variables at the point desired. It returns  the value of the input function at the desired point, and an array of partial derivative one for each inpedendent variable. Since the code for this function is not long it is listed below. It takes advantage of the AutoDiff( ) class.

    def all_derivatives(func, in_vars, point):
        '''
        Function that calculates the value of the input function at the input point and all its partial derivatives

        :params func: (function) function to evaluate whose input is a list of variables
                in_vars: (list)  list of variables to use when calculating the values of the input function
                point: (list)    list of values of each input variable at the input point

        :returns: out_val (scalar) the value of the input function at the input point
                  out_dervs (numpy array , 1 dimensional) an array of the 1st derivative values of each
                  input variable at the input point
                                                          

        :dependencies: requires class AutoDiff from module AutoDiff, package numpy imported ad np

        Example:

        f(x,y) = x^2 + y^2

        def func(vars):
            return vars[0]**2 + vars[1]**2

        a, b = all_derivatives(func, ['x','y'], [3,2])
        print(a, b)
        13.0, [6.0, 4.0]

        '''
        auto_diff_vars = []   # list to hold AutoDiff objects, one for each var in in_vars
        out_dervs =[]         # list to hold derivative values, one for each var in in_vars

        # Make AutoDiff objects
        for var, pt in zip(in_vars, point):
            var = AutoDiff(pt, 0)
            auto_diff_vars.append(var)

        # Calculate the value of func at the input point
        f = func(auto_diff_vars)
        out_val = f.val

        # Calculate the partial derivatives of func at the input point
        # Go through AutoDiff objects in auto_diff_vars
        # and during each pass change the derv value of 1 variable to 1, while others are held at 0
        # and calculate the partial derivative
        for i in range(len(auto_diff_vars)):
            auto_diff_vars[i].derv = 1
            f = func(auto_diff_vars)
            out_dervs.append(f.derv)
            auto_diff_vars[i].derv = 0

        return out_val, np.array(out_dervs)


*The multi_func_all_derivatives() function*

This function handles vector inputs and outputs. It allows the user to specify: a vector of functions, a vector of input variables, and a vectors of values defining a specific point. It the returns: a vector of output values (one for each function evaluated at the specified point), and a Jacobian matrix of partial derivatives containing partial derivatives of all functions evaluated at the specified point. Consider the equation below as an example:

\begin{equation}
\begin{pmatrix} f(x,y) \\ g(x,y)\end{pmatrix} =
\begin{pmatrix} 2xy \\ x+y^2 \end{pmatrix} at \begin{pmatrix} x=a \\ y=b \end{pmatrix}
\end{equation}


Here the multi_func_all_derivatives() function would return:  
1) out_vals, a vector of the values of each function after evaluation $(f(a,b), g(a,b))$  
2) out_derv_matrix, a matrix containing the partial derivatives evaluated at the point (a, b)

Since the code for this function is not long it is listed below. It takes advantage of the
AutoDiff( ) class and the all_derivatives( ) function:


    def multi_func_all_derivatives(functions, in_vars, point):
        '''
        Function that calculates the value of all input functions at the input point and all their
        partial derivatives producing a numpy array of output values and a numpy 2D array (matrix)
        of partial derivatives, the Jacobian

        :params func: (functions) list of function to evaluate whose input is a list of variables
                in_vars: (list)   list of variables to use when calculating the values of the input functions
                point: (list)     list of values of each input variable at the input point

        :returns: out_vals (numpy array 1 dimensional) the values of the input functions at the input point
                  out_dervs_matrix (numpy array 2 dimensional) a matrix of the 1st derivative values of each
                  input variable at the input point, the Jacobian.

        :dependencies: requires class AutoDiff from module AutoDiff, function all_derivatives from this module,
                                All_derivatives, and the package numpy imported as np

        Example:

        f(x,y) = x^2 + y^2

        (x,y) = x^2*y^2


        def func1(vars):
            return vars[0]**2 + vars[1]**2

        def func2(vars):
            return vars[0]**2*vars[1]**2

        a, b = multi_func_all_derivatives([func1,func2] , ['x','y'], [3,2])
        print(a, b)
        [13. 36.] [[ 6.  4.], [24. 36.]]

        '''
        out_vals = []
        out_dervs_matrix =[]

        # for each function in functions list
        # run all_derivatives and store output
        for func in functions:
            val, dervs = all_derivatives(func, in_vars, point)
            out_vals.append(val)
            out_dervs_matrix.append(dervs)

        return np.array(out_vals), np.array(out_dervs_matrix)



*Testing*

We have implemented a series of tests to test the AutoDiff( ) class and the all_derivatives and multi_func_all_derivatives functions. There are 67 tests in all. They are designed to be run by *pytest* and to provide full code coverage. They can be found in the modules 'test_AutoDiff.py and 'test_all_derivatives.py'. Travis CI and codecov inidicate that all tests are passed by our code, and our code coverage is very high.